In [1]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

/home/sujit25/anaconda3/envs/ml_env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_steps = 5
batch_size = 200
num_classes= 2
state_size = 4
learning_rate = 0.01

In [3]:
def generate_data(size=10000):
    X = np.array(np.random.choice(2, size))
    Y= []
    for i in range(size):
        threshold = 0.5
        if X[i-3] == 1:
            threshold += 0.5
        if X[i-8] == 1:
            threshold -= 0.25
        if np.random.rand() > threshold:
            Y.append(0)
        else:
            Y.append(1)
    return X, np.array(Y)

In [5]:
def generate_batch(raw_data, batch_size, num_steps):
    raw_x, raw_y = raw_data
    data_length = len(raw_data)
    
    # batch partition length is num of batches, but we are converting data into format of 
    # long time series -> [batch0-> batch_partition...],
    #                     [batch1-> batch_partition...],
    #                     [batch2-> batch_partition...]
    
    
    batch_partition_len = data_length//batch_size   # get no. of batches here
    
    data_x = np.zeros([batch_size, batch_partition_len] ,dtype= np.int32)
    data_y = np.zeros([batch_size, batch_partition_len] ,dtype= np.int32)
    
    for i in range(batch_size):
        data_x = raw_x[batch_partition_len*i : batch_partition_len*(i+1)]
        data_y = raw_y[batch_partition_len*i : batch_partition_len*(i+1)]        
    
    epoch_size = batch_partition_len // num_steps # calculate how many positions will be required 
    # to capture data in we fetch num_steps data units at a time
        
    # further divide batch partitions into num_steps 
    # slice vertically over data_x and data_y matrix using num_steps units horizontally
    for i in range(epoch_size):
        x = data_x[:, i* num_steps: i*(num_steps + 1)]  # x's shape - batch_size, num_steps
        y = data_y[:, i* num_steps: i*(num_steps + 1)]  # y's shape - batch_size, num_steps
        
        # yield each slice of data during each iteration
        yield (x,y)        
        
def gen_epochs(n, num_steps):
    for i in range(n):
        yield generate_batch(generate_data(),batch_size, num_steps)

In [6]:
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs_placeholder')
